# Train Model

In [1]:
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from src import datasets
from src import training
import torch
from src.architectures import *
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
import optuna
import logging
import pickle

%config Completer.use_jedi = False

# Configure Dataset
# dataset = datasets.SpecShadOcc("data/SpecShadOcc/", 9)
# test_length = 20
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset)-test_length,test_length])
# train_datasets = [datasets.SpecShadOccN(dataset, i) for i in range(1,9)]
# train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
# #train_loaders = [DataLoader(train_dataset, batch_size=2, shuffle=True) for train_dataset in train_datasets] 
# test_loader = DataLoader(test_dataset,batch_size=10)

In [2]:
data_dir = "D:DeepResidualSets/SyntheticData/"
data_dir = "D:/250x250/"
files = os.listdir(data_dir)
match = lambda x: len(re.findall("img_\d+_\d.jpg", x))== 1
cut_string = lambda x: eval(re.sub("_.*","",re.sub("img_","",x)))

files = list(filter(match,files))
files = list(map(cut_string,files))


first, last = min(files),max(files)
print(first, last)

n = last - first + 1
#train, test = train_test_split(range(first, last+1))
train, test = train_test_split(range(first, first+2000))

6462 126632


In [3]:
train_set = datasets.CGData(data_dir,train, 9)
test_set = datasets.CGData(data_dir,test, 9)

In [4]:
x,y = train_set[0]
x.size(),y.size()

(torch.Size([9, 3, 256, 256]), torch.Size([3, 256, 256]))

In [5]:
study = pickle.load( open( "study.p", "rb" ) )
study.best_params

{'agg_block': 'Attention',
 'planes': 8,
 'downsampling_factor': 2,
 'encoder_num_blocks': 10,
 'decoder_num_blocks': 10,
 'smooth_num_blocks': 10,
 'mode': 'softmax',
 'num_heads': 4,
 'dim_feedforward': 64,
 'num_layers': 2}

In [6]:
COMET_ML_PROJECT = "specshadocc-CG"
torch.cuda.empty_cache()

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
#train_loaders = [DataLoader(train_dataset, batch_size=2, shuffle=True) for train_dataset in train_datasets] 
test_loader = DataLoader(test_set,batch_size=10)
planes  = 8
downsampling_factor= 2
encoder_num_blocks = 10
decoder_num_blocks = 10
smooth_num_blocks  = 10
agg_block = "Mean"
mode = 'softmax'
num_heads= 4
dim_feedforward = 64
num_layers = 2

experiment_name = "specshadocc-" + agg_block+"-"+mode

#lr_monitor = LearningRateMonitor(logging_interval='step')

net = DeepAggNet(planes=planes, downsampling_factor=downsampling_factor,
                 encoder_num_blocks=encoder_num_blocks, 
                 decoder_num_blocks=decoder_num_blocks,
                 smooth_num_blocks=smooth_num_blocks,
                 agg_block=agg_block, mode=mode,num_heads=num_heads,
                 dim_feedforward=dim_feedforward, num_layers=num_layers)

    # Create Logger
comet_logger = CometLogger(
    api_key="tMEjeyq5M7v1IPRCvS5fyGyuo",
    workspace="semjon", # Optional
    project_name= COMET_ML_PROJECT, # Optional
    # rest_api_key=os.environ["COMET_REST_KEY"], 
    save_dir='./hyperparameter',
    experiment_name=experiment_name, # Optional,
    #display_summary_level = 0
)

#early_stop = pl.callbacks.EarlyStopping('val_loss', patience=20)
trainer = pl.Trainer(gpus=1,logger=comet_logger, max_epochs=1000, 
                     progress_bar_refresh_rate=None, 
                     fast_dev_run=False)
trainer.fit(net, train_loader, val_dataloaders=test_loader)
test_loss = trainer.test(net, test_loader)[0]["test_loss"]  

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/semjon/specshadocc-cg/3c251934c80f4af98ae958a884e30868


  | Name       | Type       | Params
------------------------------------------
0 | input      | Conv2d     | 224   
1 | output     | Conv2d     | 219   
2 | downsample | Sequential | 5.8 K 
3 | upsample   | Sequential | 5.8 K 
4 | encoder    | Sequential | 184 K 
5 | agg        | MeanAgg    | 0     
6 | decoder    | Sequential | 184 K 
7 | smooth     | Sequential | 12.8 K
------------------------------------------
394 K     Trainable params
0         Non-trainable params
394 K     Total params
1.579     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\simon\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

C:\Users\simon\AppData\Local\Continuum\anaconda3\envs\DL\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/semjon/specshadocc-cg/3c251934c80f4af98ae958a884e30868
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [6270]       : (0.0015014251694083214, 0.24372002482414246)
COMET INFO:     train_loss [1253] : (0.0015875345561653376, 0.0649876520037651)
COMET INFO:     val_loss [417]    : (0.003898763097822666, 0.04543048143386841)
COMET INFO:   Others:
COMET INFO:     Name : specshadocc-Mean-softmax
COMET INFO:   Parameters:
COMET INFO:     decoder_num_blocks  : 10
COMET INFO:     dim_feedforward     : 64
COMET INFO:  

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.003897481830790639}
--------------------------------------------------------------------------------


In [7]:
import pickle
study = pickle.load( open( "study.p", "rb" ) )

In [8]:
plot_param_importances(study)

NameError: name 'plot_param_importances' is not defined

In [ ]:
plot_edf(study)

In [ ]:
plot_contour(study, params = ["agg_block","mode","planes",""])

In [ ]:
plot_parallel_coordinate(study,params = ["agg_block","mode","planes"])


In [ ]:
import torch
import torch.nn as nn
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
memory = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_decoder(tgt, memory)
out.size()